In [0]:
from pyspark.sql.functions import col, when, datediff, current_date, coalesce, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType
import dlt
import uuid

@dlt.table(
    name="silver_crm_opportunities",
    comment="Cleansed and enriched CRM opportunities (type='opportunity') with customer area and overdue status."
)
def silver_crm_opportunities():
    leads_df = dlt.read("bronze_odoo_crm_leads")
    partners_df = dlt.read("silver_partners")
    messages_df = dlt.read("bronze_odoo_mail_messages")
    
    opportunities_df = leads_df.filter(col("type") == "opportunity")
    
    opportunities_cleaned = opportunities_df \
        .withColumn("expected_revenue", col("expected_revenue").cast(DoubleType())) \
        .withColumn("create_date", col("create_date").cast(TimestampType())) \
        .withColumn("activity_date_deadline", col("activity_date_deadline").cast(TimestampType())) \
        .withColumn("my_activity_date_deadline", col("my_activity_date_deadline").cast(TimestampType())) \
        .withColumn("probability", col("probability").cast(DoubleType())) \
        .withColumn("expected_closing_date", col("date_deadline").cast(TimestampType()))

    opportunities_enriched = opportunities_cleaned.join(
        partners_df.select("id", "customer_area"),
        opportunities_cleaned.partner_id_id == partners_df.id,
        "left"
    ).drop(partners_df.id)

    last_message_date = messages_df.groupBy("res_id").agg({"date": "max"}).alias("last_message") \
        .withColumnRenamed("max(date)", "last_message_date") \
        .withColumnRenamed("res_id", "crm_lead_id")

    opportunities_with_overdue = opportunities_enriched.join(
        last_message_date,
        opportunities_enriched.id == last_message_date.crm_lead_id,
        "left"
    )

    final_df = opportunities_with_overdue.select(
        col("id").alias("opportunity_id"),
        col("name").alias("opportunity_name"),
        col("partner_id_id").alias("partner_id"),
        col("partner_name"),
        col("customer_area"),
        col("user_id_id").alias("salesperson_id"),
        col("salesperson_name"),
        col("team_id_id").alias("sales_team_id"),
        col("sales_team_name"),
        col("stage_name").alias("stage"),
        col("expected_revenue"),
        col("probability"),
        col("create_date"),
        col("expected_closing_date")
    ).na.fill({
        "expected_revenue": 0.0,
        "probability": 0.0,
        "customer_area": "Unknown",
        "stage": "Unknown"
    })

    return final_df

In [0]:
@dlt.table(
    name="silver_crm_leads",
    comment="Cleansed and enriched CRM leads (type='lead') with customer area and overdue status, kept for potential use."
)
def silver_crm_leads():
    leads_df = dlt.read("bronze_odoo_crm_leads")
    partners_df = dlt.read("silver_partners")
    messages_df = dlt.read("bronze_odoo_mail_messages")
    
    leads_df = leads_df.filter(col("type") == "lead")

    leads_cleaned = leads_df \
        .withColumn("expected_revenue", col("expected_revenue").cast(DoubleType())) \
        .withColumn("create_date", col("create_date").cast(TimestampType())) \
        .withColumn("activity_date_deadline", col("activity_date_deadline").cast(TimestampType())) \
        .withColumn("my_activity_date_deadline", col("my_activity_date_deadline").cast(TimestampType())) \
        .withColumn("probability", col("probability").cast(DoubleType())) \
        .withColumn("expected_closing_date", 
            coalesce(col("activity_date_deadline"), col("my_activity_date_deadline")).cast(TimestampType()))

    leads_enriched = leads_cleaned.join(
        partners_df.select("id", "customer_area"),
        leads_cleaned.partner_id_id == partners_df.id,
        "left"
    ).drop(partners_df.id)

    last_message_date = messages_df.groupBy("res_id").agg({"date": "max"}).alias("last_message") \
        .withColumnRenamed("max(date)", "last_message_date") \
        .withColumnRenamed("res_id", "crm_lead_id")

    leads_with_overdue = leads_enriched.join(
        last_message_date,
        leads_enriched.id == last_message_date.crm_lead_id,
        "left"
    )

    final_df = leads_with_overdue.select(
        col("id").alias("lead_id"),
        col("name").alias("lead_name"),
        col("partner_id_id").alias("partner_id"),
        col("partner_name"),
        col("customer_area"),
        col("user_id_id").alias("salesperson_id"),
        col("salesperson_name"),
        col("team_id_id").alias("sales_team_id"),
        col("sales_team_name"),
        col("stage_name").alias("stage"),
        col("expected_revenue"),
        col("probability"),
        col("create_date"),
        col("expected_closing_date")
    ).na.fill({
        "expected_revenue": 0.0,
        "probability": 0.0,
        "customer_area": "Unknown",
        "stage": "Unknown"
    })

    return final_df

In [0]:
@dlt.table(
    name="silver_crm_field_history",
    comment="Cleansed history of field changes for CRM leads/opportunities, integrating mail.message and mail.tracking.value."
)
@dlt.expect_or_drop("valid_crm_id", "crm_id IS NOT NULL AND crm_id != -1")
def silver_crm_field_history():
    messages_df = dlt.read("bronze_odoo_mail_messages")
    tracking_df = dlt.read("bronze_odoo_field_tracking_values")
    
    field_history = tracking_df.join(
        messages_df,
        tracking_df.mail_message_id_id == messages_df.id,
        "left"
    ).withColumn("field_info", col("field_info").cast(StringType())) \
     .select(
        messages_df.res_id.alias("crm_lead_id"),
        tracking_df.mail_message_id_id.alias("mail_message_id"),
        messages_df.date.alias("change_date"),
        col("field_info"),
        tracking_df.old_value,
        tracking_df.new_value
    )

    return (
        field_history
        .withColumn("change_date", col("change_date").cast(TimestampType()))
        .na.fill({
            "field_info": "Unknown",
            "old_value": "Unknown",
            "new_value": "Unknown",
            "crm_lead_id": -1,
            "mail_message_id": -1
        })
        .withColumnRenamed("crm_lead_id", "crm_id")
    )

In [0]:
@dlt.table(
    name="silver_partners",
    comment="Cleansed partner data with standardized customer area."
)
def silver_partners():
    partners_df = dlt.read("bronze_odoo_partners")
    
    cleaned_partners = partners_df \
        .withColumn("customer_area", 
            coalesce(col("city"), col("country_name"), lit("Unknown"))) \
        .withColumn("city", col("city").cast(StringType())) \
        .withColumn("country_name", col("country_name").cast(StringType())) \
        .select(
            col("id"),
            col("name").alias("partner_name"),
            col("display_name"),
            col("is_company"),
            col("parent_id_id").alias("parent_id"),
            col("parent_company_name"),
            col("customer_area"),
            col("email"),
            col("phone")
        ).na.fill({
            "customer_area": "Unknown",
            "partner_name": "Unknown",
            "display_name": "Unknown",
            "parent_company_name": "Unknown",
            "email": "Unknown",
            "phone": "Unknown"
        })

    return cleaned_partners